In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

<span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea">EDA

### Univariant Analysis: 

In [ ]:
ds = pd.read_csv('../input/regression-with-neural-networking/concrete_data.csv');
ds.info()
ds.shape

In [ ]:
#Check for missing values
print(ds.isna().sum())
print(ds.isnull().sum())

<span style="font-family: Arial; font-weight:bold;font-size:1.5em;color:#0e92ea">
Looks the given data set is quite clean there is no missing values and all data types are numbers.

In [ ]:
#Little more statistical understanding
ds.describe().T

UNIVARIANT ANALYSIS :

1) The data set looks balanced. With 1030 number of rows and 8 columns to learn from is ok. Except the age column rest of the data looks well spread with uniform distribution.

2) slag and ash  data looks highly spreadout with min zero, 1st and 2nd Quirtile almost zero whild max value ranged from 350 to 540.

3) age data looks in days and max 365 and min 1.

### Bi-variant Analysis: 

In [ ]:
#Let's rename the columns for better handling
ds.rename(columns = {'Strength': 'strength', 'Cement': 'cement', 'Blast Furnace Slag': 'slag', 'Fly Ash': 'ash' ,'Water': 'water', 'Superplasticizer': 'superplastic', 'Coarse Aggregate': 'coarseagg' ,'Fine Aggregate': 'fineagg', 'Age': 'age' }, inplace=True)

In [ ]:
# Let's split the data before doing further analysis:
# train test split 70:30
#Remove the target varible
target = ds[['strength']]
data_set_buffer = ds.drop('strength',axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_set_buffer, target, test_size = 0.3, random_state = 123)

In [ ]:
data_set_buffer.tail()

In [ ]:
sns.relplot(x="cement", y="strength",  data=ds);
sns.relplot(x="slag", y="strength",  data=ds);
sns.relplot(x="ash", y="strength",  data=ds);
sns.relplot(x="water", y="strength",  data=ds);
sns.relplot(x="superplastic", y="strength",  data=ds);
sns.relplot(x="coarseagg", y="strength",  data=ds);
sns.relplot(x="fineagg", y="strength",  data=ds);
sns.relplot(x="age", y="strength",  data=ds);
#Looks like superplastic and cement have +ve correlation with the strength rest all are not straight forward to infer.

In [ ]:
sns.relplot(x="cement", y="strength", alpha=.5, palette="muted",
            height=6, data=ds)
#distplot
sns.relplot(x="water", y="strength", alpha=.5, palette="muted",
            height=6, data=ds)

#histogram
sns.relplot(x="slag", y="strength", alpha=.5, palette="muted",
            height=6, data=ds)

sns.relplot(x="ash", y="strength", alpha=.5, palette="muted",
            height=6, data=ds)


*It looks like the strength of cement depends on various proportions of ingredient. For e.g. The strength remains same for various amoung of ash level. Same behaviour can be observed for Slag ans Super plasitc. But one thing is clear that the strength has linearly related with the main ingredient that is 'Cement'. One important thing like any chemical composition, there need to be a perfect balance of various reagents required for best strength to achieve.This paves the way to do multivariant analysis. For e.g. we might observe that, if we have more water, we might need more fly ash to strengthen the concrete. Let's find out this.*

## Multi-Variant Analysis:

In [ ]:
# Let's analyse the strength against water and fly-ash level

# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot
ax.scatter(ds.strength,ds.ash,  color="red", marker=",")
# set x-axis label
ax.set_xlabel("Strength",fontsize=14)
# set y-axis label
ax.set_ylabel("Ash",color="red",fontsize=14)

# twin object for two different y-axis on the sample plot
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.scatter(ds.strength,ds.water,color="blue",marker="+")
ax2.set_ylabel("Water",color="blue",fontsize=14)
plt.show()
# save the plot as a file
sns.relplot(x="water", y="ash", alpha=.5, palette="muted",
            height=6, data=ds)

Looks like yes. We might need more water for more ash.

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(ds.loc[:],figsize=(12, 12),diagonal="kde")
plt.tight_layout()
plt.show()

### Observation :
There is no fixed ratio and proportion of input variables to achive the higher strength. 

We can also see that, The parameters like water vs superplastic follows an L shaped curve. That shows we need to maintain the ratios of multiple ingredients to achieve the same strength. 

We can  observe ash vs slag follows a linear relation. i.e. we have multi-colinearity relation among the input parameters which we must have to take care. 

Except cement and Strength other variables have multiple Gaussian distributions So definitely they have outliers also. 

Scaling : Looks fine as we can see the input parameters with  outliers and we have independent variables in same units.
 

In [ ]:
ds.corr().T

### Observation : 
water has -ve correlation with superplastic and fineagg.
cement and strength has +ve correlation.

### Let's address the presence outliers and missing values and perform necessary imputation 

In [ ]:
#Strength is the target column while rest are our input variable.
#Let's do box plot to find the outliers if there any
plt.figure(figsize=(35,15))
sns.boxplot(data=ds)

In [ ]:
import itertools
plt.figure(figsize=(10,20))
j=0;
for i, c in zip(ds.columns , list(ds.columns.values) ):
    j=j+1
    plt.subplot(3,3,j)
    sns.swarmplot( y = ds[i]);
    plt.title= c
    plt.show

### Observation:
Looks like there are not much outliers but age,water, slag and superplastic surely have some outlier. If these belong to feature importance we have to take care of it.
There are data for slag and ash also looks quite right skewed as we have analysed earlier.

In [ ]:
# Let's replace the outliers with median values
import numpy as np
outliers =  []
from scipy import stats
zscore = ds.apply(stats.zscore)
ds_columns = ds.columns.values
for key, value in zscore.iteritems():
    row=0
    medianV = np.median(ds[[key]])
    columnnum = ds.columns.get_loc(key)
    for v in value:
        if v>3 or v < -3 :
            outliers.append(v)
            ds.iloc[row,columnnum] = medianV
            print (ds.iloc[row,columnnum] )
        row = row + 1
     
print("outlier size ", len(outliers))

### There are 10 values with zscore inbetween -3 and +3 so we got 10 outliers. 

###  Feature Engineering , Model Building and Model Tuning

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr_model = DecisionTreeRegressor()
dtr_model.fit(X_train , y_train)

In [ ]:
print('The feature importances starting from hightest:')
for (i, j) in zip(X_train.columns , reversed(sorted(dtr_model.feature_importances_))): 
    print(i,j)

### Observation : cement,age ,water , ash , slag are important feature of imprtance

In [ ]:
print(dtr_model.score(X_train, y_train))
print(dtr_model.score(X_test, y_test))

#### Looks like the Decision Tree Regression model is in over fitting state we have to tune our model.

In [ ]:
# Regularizing the Decision tree classifier to avoid over fitting situation described above
reg_dtr_model = DecisionTreeRegressor( max_depth = 5,random_state=1,min_samples_leaf=6)
reg_dtr_model.fit(X_train, y_train)

In [ ]:
print(reg_dtr_model.score(X_train, y_train))
print(reg_dtr_model.score(X_test, y_test))

In [ ]:
print('The feature importances:')
for (i, j) in zip(X_train.columns ,reversed(sorted(reg_dtr_model.feature_importances_))): 
    print(i,j)

In [ ]:
import matplotlib.pyplot as plt

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = X_train.columns
sizes = reg_dtr_model.feature_importances_
fig1, ax1 = plt.subplots()
wedges, texts, autotexts = ax1.pie(sizes, labels=labels, autopct='%1.01f%%',
        shadow=True, startangle=90,pctdistance=1.2, labeldistance=1.5 )
ax1.axis('equal') 
ax1.legend(wedges, labels,
          title="Feature Importance:",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1))
plt.setp(autotexts, size=10, weight="bold")

ax1.set_title("Feature Importance Pie")

### Observation : Cement,Age,Slag and Water Can  explain more than 96% of Strength. We can keep 4 variables and drop the rest. Just before doing let's find if there any strong correlation between other dependant variables.

In [ ]:
#Let's confirm the correlation with correlation matrix
#Find correlation
plt.figure(figsize=(10,10))
corr =train_data_Set.corr()
matrix = np.triu(corr)
ax=sns.heatmap(corr, annot=True, mask=matrix,cmap= sns.diverging_palette(20, 220, n=200), linewidths=2,linecolor='white',square=True)
plt.show()

**Observation : Water has strong -ve relation with Superplastic and fineagg so just considering water and not taking Superplastic and fineagg will not be a good idea.
Similarly, ash has -ve co-relation with cement so we can't drop ash too.**

### Observation : Our feature of imprtance would be : Cement,Age,Slag, Water, Superplastic,fineagg and ash

In [ ]:
#Let's drop the coarseagg column and use important features only to analyse further.
data_set_buffer_feature_imprtance = ds.drop('coarseagg',axis=1)
X_train, X_test, y_train, y_test = train_test_split(data_set_buffer_feature_imprtance, target, test_size = 0.3, random_state = 123)

### Let's try a simple Linear Regression and test with cross validataion to arrive any conclusion.

In [ ]:
import statsmodels.api as sm
lr_1 = sm.OLS(y_train, X_train).fit()

In [ ]:
y_pred = lr_1.predict(X_test)

In [ ]:
from sklearn import metrics
metrics.explained_variance_score(y_test, y_pred)

**Linear Regression is almost perfect! Let's find out with cross validation.**

K fold cross validation without removing outlier and with 50 folds:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# Import Linear Regression machine learning library
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
num_folds = 50
seed = 7

kfold = KFold(n_splits=num_folds, random_state=seed)
model = LinearRegression()
results = cross_val_score(model, data_set_buffer, target, cv=kfold)
print(results)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))
print(color.RED)
print(color.BOLD)
print('Accuracy Score')
print(color.BLUE)
print('Average: ', results.mean())
print('Standard deviation ',results.std())

In [ ]:
data_set_buffer_removed_outlier = data_set_buffer
zscore = data_set_buffer_removed_outlier.apply(stats.zscore)
ds_columns = data_set_buffer_removed_outlier.columns.values
for key, value in zscore.iteritems():
    row=0
    medianV = np.median(data_set_buffer[[key]])
    columnnum = data_set_buffer_removed_outlier.columns.get_loc(key)
    for v in value:
        if v>3 or v < -3 :
            data_set_buffer_removed_outlier.iloc[row,columnnum] = medianV
        row = row + 1

target_removed_outlier = target
zscore = target_removed_outlier.apply(stats.zscore)
ds_columns = target_removed_outlier.columns.values
for key, value in zscore.iteritems():
    row=0
    medianV = np.median(target[[key]])
    columnnum = target_removed_outlier.columns.get_loc(key)
    for v in value:
        if v>3 or v < -3 :
            target_removed_outlier.iloc[row,columnnum] = medianV
        row = row + 1


**Let's apply paarameter tuning**

In [ ]:
num_folds = 6
seed = 7

kfold = KFold(n_splits=num_folds, random_state=seed)
model = LinearRegression()
results = cross_val_score(model, data_set_buffer_removed_outlier, target_removed_outlier, cv=kfold)
print(results)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))
print(color.RED)
print(color.BOLD)
print('Accuracy Score after replacing outliers with median')
print(color.BLUE)
print('Average: ', results.mean())
print('Standard deviation ',results.std())

Observation : Our accuracy seems sensitive to outliers here as we can see with 6 K fold cross validation.

# Create a regularized RIDGE model and note the coefficients

In [ ]:
ridge = Ridge(alpha=.3)
ridge.fit(X_train,y_train)
print ("Ridge model:", (ridge.coef_))

# Create a regularized LASSO model and note the coefficients

In [ ]:
lasso = Lasso(alpha=0.1)
lasso.fit(X_train,y_train)
print ("Lasso model:", (lasso.coef_))

## Let us compare their scores

In [ ]:
print(lasso.score(X_train, y_train))
print(lasso.score(X_test, y_test))

In [ ]:
print(ridge.score(X_train, y_train))
print(ridge.score(X_test, y_test))

# Let us generate polynomial models reflecting the non-linear interaction between some dimensions

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

X_scaled = preprocessing.scale(data_set_buffer_removed_outlier)
X_scaled = pd.DataFrame(X_scaled, columns=data_set_buffer_removed_outlier.columns)  

y_scaled = preprocessing.scale(y_train)
y_scaled = pd.DataFrame(y_scaled, columns=target.columns)  

poly = PolynomialFeatures(degree = 2, interaction_only=True)
X_poly = poly.fit_transform(X_scaled)
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(X_poly, target, test_size=0.30, random_state=1)
X_train_poly.shape

from sklearn.linear_model import LinearRegression
regression_model = LinearRegression()
regression_model.fit(X_train_poly, y_train_poly)
print(regression_model.coef_[0])
ridge = Ridge(alpha=.3)
ridge.fit(X_train_poly,y_train_poly)
print ("Ridge model:", (ridge.coef_))

In [ ]:
print(ridge.score(X_train_poly, y_train_poly))
print(ridge.score(X_test_poly, y_test_poly))


In [ ]:
lasso = Lasso(alpha=0.01)
lasso.fit(X_train_poly,y_train_poly)
print ("Lasso model:", (lasso.coef_))
print(lasso.score(X_train_poly, y_train_poly))
print(lasso.score(X_test_poly, y_test_poly))

**OBSERVATION : As simple linear regression is providing high accuracy we DON'T NEED polynomial of higher order. We shall be fine with simple linear models. As we observed with higher order, we loose our predectibility power.**

Important Point : We have to employ only Regression technique and not classificiation ones because our output or the target is continious variable. Classification is ONLY be used for categorical varibles. As we have a data set of where target variable is continuous neiture, Regression would do good rather than classification Techniques such as Naive Bayes or KNN. Let's see

But our Kfold cross validation is only Accuracy 70%. That shows our model is succeptible to outliers and not generalizing well also.

### Linear Regression

In [ ]:
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)
y_pred = regression_model.predict(X_test)
from sklearn import metrics
metrics.explained_variance_score(y_test, y_pred)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
randomf = RandomForestRegressor(n_estimators=10, max_depth = 3)
randomf.fit(X_train, y_train)
randomf.score(X_test, y_test)

### Gradient Boosting Regressor 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)
gbr.score(X_test, y_test)

### KNeighborsRegressor 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knnr = KNeighborsRegressor(n_neighbors=10)
knnr.fit(X_train, y_train)
knnr.score(X_test, y_test)

<span style="font-family: Arial; font-weight:bold;font-size:1.9em;color:#0e92ea">Conclusion

### 1) With Linear Regression , Random Forest and Gradient Boosting Regressor we can achieve > 95% accuracy while KNeighbors Regressor has only 76% accuracy.
### 2) This Proves our earlier assumption that the given dataset fits well for regressor algos which works well for continuous data rather than classification techniques.
### 3) Our model accuracy improved significantly after replacing outliers with corresponding medians. i.e. the models we have used or infact most of the models are succeptible to  outliers present in feature importance input data.
### 4) Polynomials of higher orders are not necessarily improve model accuracy or generalization. It depends the dataset. As we observed this data set the strength of concrete surely depends on various proportions of inputs e.g. amount of cement, amount of fly ash, age so linear equations or first degree equations works well. e.g. =>3x+6y+6z = 8s